In [1]:
import sys
import pyccl as ccl
import pickle
import camb
sys.path.insert(0,'./')
# from importlib import reload

In [2]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
c=LocalCluster(n_workers=1,processes=False,memory_limit='25gb',threads_per_worker=14,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(c)

In [3]:
from cov_3X2 import *
from lsst_utils import *

In [4]:
#only for python3
import importlib
reload=importlib.reload

In [5]:
zs=lsst_source_tomo_bins()

ns0:  27.0


In [6]:
zs['zmax']

4.0

# Power Spectrum covariance

In [7]:
#setup parameters
lmax_cl=5000
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=40
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell

do_cov=True
bin_cl=True

SSV_cov=True
tidal_SSV_cov=False
Tri_cov=True

bin_xi=True
theta_bins=np.logspace(np.log10(1./60),1,20)

##  Test with 1 redshift bin

In [8]:
use_window=False
do_cov=False

In [9]:
z0=1 #1087
zs_bin1=source_tomo_bins(zp=[z0],p_zp=np.array([1]),ns=30,use_window=False,)

In [10]:
#use all ell
kappa0=cov_3X2(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       )

./cov_utils.py:60: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th


{('shear', 'shear'): [(0, 0)]} {'shear': 1, 'galaxy': 0, 'kappa': 0}
Win gen: step size 807
Window done


In [11]:
kappa0.Ang_PS.PS.pk_func

<bound method Power_Spectra.class_pk of <power_spectra.Power_Spectra object at 0x7fd2f8437a90>>

In [12]:
kappa0.Ang_PS.PS.class_pk(z=[0])

(array([[1.31144486e+03, 1.31434952e+03, 1.31726055e+03, ...,
         9.05451097e-01, 9.00827843e-01, 8.96227943e-01]]),
 array([3.00000000e-04, 3.00691710e-04, 3.01385015e-04, ...,
        2.98621350e+01, 2.99309881e+01, 3.00000000e+01]))

In [ ]:
%time cl0G=kappa0.cl_tomo() 



In [ ]:
%time cl0=cl0G['stack'].compute()